# ATS Routing Graph Builder

In this version (0.1), we use
- ATS graph from ATS.txt (the traditional routing options)
- DCT edges from EUROCONTROL Appendix B3 - DCT

The DCT links usage is under conditions. We **do not consider** them in this version. Perhaps a future version will implement a more advanced search algorithm that takes into account these constraints.

We **do not consider FRA**. Use version 0.2 if we want to incorporate FRA routing options.

In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
import os
import sys
load_dotenv()

PROJECT_ROOT = os.getenv('PROJECT_ROOT')

# Add PROJECT_ROOT to the sys.path
sys.path.append(PROJECT_ROOT)


In [2]:
import networkx as nx
from tqdm import tqdm
import os

def is_in_europe(lat, lon):
    """
    Checks if a given latitude and longitude fall within an approximate bounding
    box for the European continent.
    
    Europe is approximated as:
      Latitude: 30°N to 72°N
      Longitude: -25°E to 45°E
    """
    return 30 <= lat <= 72 and -25 <= lon <= 45

In [3]:
import random

def generate_random_two_digits():
    return str(random.randint(0, 99)).zfill(2)



In [4]:
def build_graph_from_ats(filename, bidirectional=False):
    """
    Builds a graph from an ATS file containing airway segments.
    
    The ATS file is expected to have header lines for each airway (starting with "A")
    and segment lines (starting with "S"). For example:
    
      A,A1,46
      S,KEC,33.447742,135.794494,ALBAT,33.364503,135.441514,0,262,18.37
      S,ALBAT,33.364503,135.441514,HALON,33.248769,134.997222,260,262,23.34
      ...
      
    For each "S" (segment) line, this function:
      - Parses the start and end fixes and their coordinates.
      - Filters out segments if either endpoint is not within Europe.
      - Adds fixes (nodes) to the graph.
      - Adds an edge for the segment with attributes for minimum altitude,
        maximum altitude, distance, and the airway name.
    
    Parameters:
      filename (str): Path to the ATS data file.
      bidirectional (bool): If True, add edges in both directions.
    
    Returns:
      networkx.DiGraph: The directed graph representing ATS fixes and segments.
    """
    # Create a directed graph; convert later to undirected if needed.
    G = nx.DiGraph()
    current_airway = "Unknown"  # Default airway name if header hasn't been seen

    with open(filename, 'r') as file:
        for line in file:
            line = line.strip()
            if not line or line.startswith("#"):
                continue  # Skip empty or comment lines

            parts = line.split(',')
            record_type = parts[0].strip().upper()

            # If the line is an airway header, update the current airway name.
            if record_type == "A":
                if len(parts) >= 2:
                    current_airway = parts[1].strip()
                    # Append two random digits to the airway name to make it unique
                    current_airway = current_airway
                continue


            # Process only segment lines.
            if record_type != "S":
                continue

            if len(parts) < 10:
                print(f"Skipping malformed line: {line}")
                continue

            # Unpack fields from the segment line.
            # Format: S, start_fix, start_lat, start_lon, end_fix, end_lat, end_lon, min_alt, max_alt, distance
            (_, start_fix, start_lat, start_lon, end_fix, end_lat, end_lon,
             min_alt, max_alt, distance) = parts

            # Convert coordinate and numeric fields.
            try:
                start_lat = float(start_lat)
                start_lon = float(start_lon)
                end_lat   = float(end_lat)
                end_lon   = float(end_lon)
                min_alt   = float(min_alt)
                max_alt   = float(max_alt)
                distance  = float(distance)
            except ValueError:
                print(f"Skipping line due to conversion error: {line}")
                continue

            # Filter: Only include segments with both endpoints in Europe.
            if not (is_in_europe(start_lat, start_lon) and is_in_europe(end_lat, end_lon)):
                # Uncomment the next line to see which segments are skipped.
                # print(f"Skipping non-European segment: {start_fix} -> {end_fix}")
                continue

            # Add the nodes (fixes) with their coordinates.
            if start_fix not in G:
                G.add_node(start_fix, lat=start_lat, lon=start_lon,
                           type='ATS', refs='') # refs: to provide pointers to other nodes e.g., NIK -> NIK_22, NIK_86
            else:
                # Start fix already exists, check if coordinates match
                if abs(G.nodes[start_fix]['lat'] - start_lat) > 1e-4 or abs(G.nodes[start_fix]['lon'] - start_lon) > 1e-4:
                    print(f"Start fix {start_fix} has different coordinates: {G.nodes[start_fix]['lat']}, {G.nodes[start_fix]['lon']} != {start_lat}, {start_lon}")
                    original_start_fix = start_fix
                    start_fix = start_fix + '_' + generate_random_two_digits()
                    G.add_node(start_fix, lat=start_lat, lon=start_lon, type='ATS', refs='')
                    # Modify the refs of the original node
                    G.nodes[original_start_fix]['refs'] = G.nodes[original_start_fix]['refs'] + f'{start_fix}, '
            if end_fix not in G:
                G.add_node(end_fix, lat=end_lat, lon=end_lon, type='ATS', refs='')
            else:
                # End fix already exists, check if coordinates match
                if abs(G.nodes[end_fix]['lat'] - end_lat) > 1e-4 or abs(G.nodes[end_fix]['lon'] - end_lon) > 1e-4:
                    print(f"End fix {end_fix} has different coordinates: {G.nodes[end_fix]['lat']}, {G.nodes[end_fix]['lon']} != {end_lat}, {end_lon}")
                    original_end_fix = end_fix
                    end_fix = end_fix + '_' + generate_random_two_digits()
                    G.add_node(end_fix, lat=end_lat, lon=end_lon, type='ATS', refs='')
                    # Modify the refs of the original node
                    G.nodes[original_end_fix]['refs'] = G.nodes[original_end_fix]['refs'] + f'{end_fix}, ' 

            # Add the edge (segment) with its attributes, including the airway name.
            edge_attrs = {
                'distance': distance,
                'airway': current_airway,
                'edge_type': 'airway'
            }
            G.add_edge(start_fix, end_fix, **edge_attrs)

            # Optionally, add the reverse edge if the segment is bidirectional.
            if bidirectional:
                G.add_edge(end_fix, start_fix, **edge_attrs)

    return G

In [5]:
ats_file = os.path.join(PROJECT_ROOT, "data", "airac", "ATS.txt")

# Set bidirectional=True if segments are used in both directions.
graph_ats = build_graph_from_ats(ats_file, bidirectional=False)

print(f"Number of fixes (nodes): {graph_ats.number_of_nodes()}")
print(f"Number of route segments (edges): {graph_ats.number_of_edges()}")

End fix BNA has different coordinates: 36.651297, 3.591522 != 32.124444, 20.253611
Start fix BNA has different coordinates: 36.651297, 3.591522 != 32.124444, 20.253611
End fix BNA has different coordinates: 36.651297, 3.591522 != 32.124444, 20.253611
Start fix BNA has different coordinates: 36.651297, 3.591522 != 32.124444, 20.253611
End fix BNA has different coordinates: 36.651297, 3.591522 != 32.124444, 20.253611
Start fix BNA has different coordinates: 36.651297, 3.591522 != 32.124444, 20.253611
End fix BNA has different coordinates: 36.651297, 3.591522 != 32.124444, 20.253611
Start fix BNA has different coordinates: 36.651297, 3.591522 != 32.124444, 20.253611
End fix BNA has different coordinates: 36.651297, 3.591522 != 32.124444, 20.253611
Start fix BNA has different coordinates: 36.651297, 3.591522 != 32.124444, 20.253611
End fix BOURI has different coordinates: 31.69, 18.716389 != 37.989167, 5.0
Start fix BOURI has different coordinates: 31.69, 18.716389 != 37.989167, 5.0
End fi

In [6]:
def collapse_duplicate_nodes(G):
    """
    Collapses nodes that reference each other and have virtually identical coordinates.
    
    Args:
        G (networkx.Graph): Input graph
        
    Returns:
        networkx.Graph: Graph with duplicate nodes collapsed
    """
    # Copy the graph first
    G = G.copy()
    nodes_removed = 0
    
    # Make a copy to avoid modifying graph during iteration
    nodes = list(G.nodes(data=True))
    
    # Track nodes that have been merged to avoid repeat processing
    merged = set()
    
    for node, data in nodes:
        if node in merged:
            continue
            
        # Skip if no refs field or empty
        if 'refs' not in data or not data['refs']:
            continue
            
        # Get referenced nodes
        refs = data['refs'].split(',')
        refs = [r.strip() for r in refs if r.strip()]
        # Remove empty strings from refs
        refs = [r for r in refs if r]
        # Add the current node to refs for consideration
        refs.append(node)
            
        # Get coordinates for all related refs
        ref_coords = {}
        for r in refs:
            if r in G and 'lat' in G.nodes[r] and 'lon' in G.nodes[r]:
                ref_coords[r] = (float(G.nodes[r]['lat']), float(G.nodes[r]['lon']))
        
        # Group refs by identical coordinates
        coord_groups = {}
        for r, coords in ref_coords.items():
            found = False
            for group_coords, group_refs in coord_groups.items():
                if (abs(coords[0] - group_coords[0]) < 1e-4 and 
                    abs(coords[1] - group_coords[1]) < 1e-4):
                    group_refs.append(r)
                    found = True
                    break
            if not found:
                coord_groups[coords] = [r]
        
        # Keep only first ref from each coordinate group
        refs_to_keep = list(set([group[0] for group in coord_groups.values()]))
        print(f'Refs to keep: {refs_to_keep}')
        refs_to_remove = list(set([r for r in refs if r not in refs_to_keep]))
        
        for refrm in refs_to_remove:
            print(f'Attempting to remove {refrm}')
            print(f'Refs to remove: {refs_to_remove}')
            # Find the ref in refs_to_keep that have the same coordinates
            # Find ref with matching coordinates in refs_to_keep
            ref_coords = (float(G.nodes[refrm]['lat']), float(G.nodes[refrm]['lon']))
            for refkeep in refs_to_keep:
                keep_coords = (float(G.nodes[refkeep]['lat']), float(G.nodes[refkeep]['lon']))
                if (abs(ref_coords[0] - keep_coords[0]) < 1e-4 and 
                    abs(ref_coords[1] - keep_coords[1]) < 1e-4):
                    rk = refkeep
                    break
            # Redirect all edges from ref node to original node
            for pred in G.predecessors(refrm):
                edge_data = G.get_edge_data(pred, refrm)
                G.add_edge(pred, rk, **edge_data)
                
            for succ in G.successors(refrm):
                edge_data = G.get_edge_data(refrm, succ)
                G.add_edge(rk, succ, **edge_data)
            
            # Remove the duplicate node
            print(f'Removing node {refrm}')
            G.remove_node(refrm)

            merged.add(refrm)
            nodes_removed += 1
                
    print(f"Removed {nodes_removed} duplicate nodes")

    print(f'Revising refs properties...')
    # Revise the refs property
    # Update refs property to only include existing nodes
    for node in G.nodes():
        refs_str = G.nodes[node].get('refs', '')
        if refs_str:
            # Split refs string into list and remove empty strings
            refs = [r.strip() for r in refs_str.split(',') if r.strip()]
            
            # Filter to only keep refs that exist in graph
            existing_refs = [r for r in refs if r in G.nodes]
            
            # Update the refs property with filtered list
            G.nodes[node]['refs'] = ', '.join(existing_refs) if existing_refs else ''
    
    return G

In [7]:
graph_ats_no_dup = collapse_duplicate_nodes(graph_ats)

Refs to keep: ['BRY', 'BRY_99']
Attempting to remove BRY_90
Refs to remove: ['BRY_90', 'BRY_71', 'BRY_24', 'BRY_30', 'BRY_60', 'BRY_48', 'BRY_67', 'BRY_39', 'BRY_09', 'BRY_41']
Removing node BRY_90
Attempting to remove BRY_71
Refs to remove: ['BRY_90', 'BRY_71', 'BRY_24', 'BRY_30', 'BRY_60', 'BRY_48', 'BRY_67', 'BRY_39', 'BRY_09', 'BRY_41']
Removing node BRY_71
Attempting to remove BRY_24
Refs to remove: ['BRY_90', 'BRY_71', 'BRY_24', 'BRY_30', 'BRY_60', 'BRY_48', 'BRY_67', 'BRY_39', 'BRY_09', 'BRY_41']
Removing node BRY_24
Attempting to remove BRY_30
Refs to remove: ['BRY_90', 'BRY_71', 'BRY_24', 'BRY_30', 'BRY_60', 'BRY_48', 'BRY_67', 'BRY_39', 'BRY_09', 'BRY_41']
Removing node BRY_30
Attempting to remove BRY_60
Refs to remove: ['BRY_90', 'BRY_71', 'BRY_24', 'BRY_30', 'BRY_60', 'BRY_48', 'BRY_67', 'BRY_39', 'BRY_09', 'BRY_41']
Removing node BRY_60
Attempting to remove BRY_48
Refs to remove: ['BRY_90', 'BRY_71', 'BRY_24', 'BRY_30', 'BRY_60', 'BRY_48', 'BRY_67', 'BRY_39', 'BRY_09', 'BR

In [8]:
# Save the graph_ats to a GraphML file for later use
import networkx as nx
import os

# Ensure the output directory exists
output_dir = os.path.join(PROJECT_ROOT, "data", "graphs")
os.makedirs(output_dir, exist_ok=True)

# Define the output file path
output_file = os.path.join(output_dir, "graph_ats.graphml")

# Write the graph to a GraphML file
nx.write_graphml(graph_ats_no_dup, output_file)

print(f"Graph saved to {output_file}")



Graph saved to E:/project-akrav\data\graphs\graph_ats.graphml


In [9]:
import pandas as pd
import os
rad_dct_df = pd.read_csv(os.path.join(PROJECT_ROOT, "data", "rad", "RAD_DCT.csv"))
rad_dct_df.columns = ["ChangeInd","ValidFrom","ValidUntil","ID","From","To","LowerVertLimit","UpperVertLimit","Available","Utilization","DCTTimeAvail","OpGoal","Remarks","CruisingLevelsDir","ATCUnit","ReleaseDate","SpecialEvent"]
rad_dct_df.head(1)

,ChangeInd,ValidFrom,ValidUntil,ID,From,To,LowerVertLimit,UpperVertLimit,Available,Utilization,DCTTimeAvail,OpGoal,Remarks,CruisingLevelsDir,ATCUnit,ReleaseDate,SpecialEvent
0,AMD,06 FEB 2025 [2501],UFN,EG5256,MIMVA,OTBED,105,660,Yes,"NOT AVBL FOR TFC\nDEP EHAAFIR EXC DEP (EHBK, E...",H24\n----------\nH24,To provide connectivity for traffic at Maastri...,NaN,NaN,EG**ACC,05 FEB 2025,NaN


In [10]:
from utils.haversine import haversine_distance

def build_graph_from_rad_dct3(rad_dct_df, graph_ats):
    graph_ats = graph_ats.copy()
    # Filter for available DCT routes
    available_dct = rad_dct_df[rad_dct_df['Available'] == 'Yes']
    
    # Initialize counter for successfully created edges
    edges_created = 0
    total_available = len(available_dct)
    
    # Create edges for each available DCT route
    for _, row in available_dct.iterrows():
            # Add edge if both nodes exist
            from_fix = row['From'].strip().upper()
            to_fix = row['To'].strip().upper()
            if from_fix in graph_ats.nodes and to_fix in graph_ats.nodes:
                # Get the lat and lon of the nodes
                from_lat = graph_ats.nodes[from_fix]['lat']
                from_lon = graph_ats.nodes[from_fix]['lon']
                to_lat = graph_ats.nodes[to_fix]['lat']
                to_lon = graph_ats.nodes[to_fix]['lon']
                # Compute the distance using haversine distance
                distance = haversine_distance(from_lat, from_lon, to_lat, to_lon)
                if distance > 20:
                     # Attention: we can use the create_edge_with_closest_refs function from nav_graph.py as well (they are the same thing)
                     print(f'DCT link between {from_fix} and {to_fix} is {distance:.2f}nm. Considering alternatives...')
                     from_fix_refs = graph_ats.nodes[from_fix]['refs']
                     to_fix_refs = graph_ats.nodes[to_fix]['refs']
                     # Split refs strings into lists
                     from_fix_refs = from_fix_refs.split(',') if from_fix_refs else []
                     to_fix_refs = to_fix_refs.split(',') if to_fix_refs else []
                     # Add the current fixes to the refs
                     from_fix_refs.append(from_fix)
                     to_fix_refs.append(to_fix)
                     # Find the closest pair of refs
                     min_distance = float('inf')
                     best_from_ref = None 
                     best_to_ref = None
                     
                     for from_ref in from_fix_refs:
                         if from_ref not in graph_ats.nodes:
                             continue
                         for to_ref in to_fix_refs:
                             if to_ref not in graph_ats.nodes:
                                 continue
                             # Get coordinates
                             from_ref_lat = graph_ats.nodes[from_ref]['lat']
                             from_ref_lon = graph_ats.nodes[from_ref]['lon']
                             to_ref_lat = graph_ats.nodes[to_ref]['lat']
                             to_ref_lon = graph_ats.nodes[to_ref]['lon']
                             
                             # Calculate simple distance (absolute difference)
                             dist = abs(from_ref_lat - to_ref_lat) + abs(from_ref_lon - to_ref_lon)
                             
                             if dist < min_distance:
                                 min_distance = dist
                                 best_from_ref = from_ref
                                 best_to_ref = to_ref

                     # Recalculate the distance in nm
                     best_min_distance = haversine_distance(
                         graph_ats.nodes[best_from_ref]['lat'], 
                         graph_ats.nodes[best_from_ref]['lon'], 
                         graph_ats.nodes[best_to_ref]['lat'], 
                         graph_ats.nodes[best_to_ref]['lon']
                     )
                     
                     # Create edge between closest refs if found
                     if best_from_ref and best_to_ref:
                         if not graph_ats.has_edge(best_from_ref, best_to_ref):
                             graph_ats.add_edge(best_from_ref, best_to_ref,
                                              distance=best_min_distance,
                                              airway='',
                                              edge_type='DCT')
                             print(f'Established link between {best_from_ref} and {best_to_ref} instead. New distance is {best_min_distance}')
                             edges_created += 1
                    
                     continue
                # Check if edge already exists
                if graph_ats.has_edge(from_fix, to_fix):
                    continue
                graph_ats.add_edge(from_fix, to_fix,
                                   distance=distance,
                                   airway='',
                                   edge_type='DCT')
                edges_created += 1
            else:
                if from_fix not in graph_ats.nodes:
                    print(f"From node {from_fix} not found in graph_ats. Probably outside Europe.")
                if to_fix not in graph_ats.nodes:
                    print(f"To node {to_fix} not found in graph_ats. Probably outside Europe.")
            
    print(f"Created {edges_created} DCT edges out of {total_available} available DCT routes")
    
    return graph_ats


In [11]:
graph_ats_dct3 = build_graph_from_rad_dct3(rad_dct_df, graph_ats_no_dup)

DCT link between MIMVA and OTBED is 118.21nm. Considering alternatives...
Established link between MIMVA and OTBED instead. New distance is 118.20740866297902
DCT link between LAKEY and MAMUL is 77.86nm. Considering alternatives...
Established link between LAKEY and MAMUL instead. New distance is 77.85637800351923
DCT link between MAMUL and MOGLI is 76.04nm. Considering alternatives...
Established link between MAMUL and MOGLI instead. New distance is 76.04341072449706
To node ODVOD not found in graph_ats. Probably outside Europe.
DCT link between MARJA and BAGBO is 286.59nm. Considering alternatives...
Established link between MARJA and BAGBO instead. New distance is 286.58552844382933
DCT link between MARJA and BAGBO is 286.59nm. Considering alternatives...
DCT link between BAGBO and MARJA is 286.59nm. Considering alternatives...
Established link between BAGBO and MARJA instead. New distance is 286.58552844382933
DCT link between BAGBO and MARJA is 286.59nm. Considering alternatives..

In [12]:
# Write the graph_ats_dct3 to a GRAPHML file
# Write the graph to a GRAPHML file
import networkx as nx
nx.write_graphml(graph_ats_dct3, os.path.join(PROJECT_ROOT, "data", "graphs", "route_graph_dct3.graphml"))
print("Graph written to data/graphs/route_graph_dct3.graphml")


Graph written to data/graphs/route_graph_dct3.graphml


In [13]:
def generate_airport_csv(filename):
    """
    Extracts airport information from Airport.txt and saves it to a CSV file.
    
    Airport line format:
    A,ICAO,NAME,LAT,LON,ELEV,LENGTH,UNUSED1,UNUSED2,UNUSED3
    
    Parameters:
        filename (str): Path to the Airport.txt file
    """
    import csv
    import os
    
    output_file = os.path.join(os.path.dirname(filename), 'airports.csv')
    
    # Define CSV headers
    headers = ['icao', 'name', 'latitude', 'longitude', 'elevation']
    
    with open(filename, 'r') as infile, open(output_file, 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(headers)
        
        for line in infile:
            line = line.strip()
            if not line or not line.startswith('A'):
                continue
                
            parts = line.split(',')
            if len(parts) < 5:
                continue
                
            # Extract relevant fields
            icao = parts[1]
            name = parts[2]
            try:
                lat = float(parts[3])
                lon = float(parts[4])
                elev = float(parts[5])
            except (ValueError, IndexError):
                print(f"Skipping malformed line: {line}")
                continue
                
            writer.writerow([icao, name, lat, lon, elev])
    
    print(f"Airport data saved to: {output_file}")

# Usage:
airport_file = os.path.join(PROJECT_ROOT, "data", "airac", "Airports.txt")
generate_airport_csv(airport_file)

Airport data saved to: E:/project-akrav\data\airac\airports.csv
